In [1]:
import gzip
import os 
import json
import re

import numpy as np
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

from dotenv import load_dotenv

from utils import config
from utils import utilities

In [2]:
conn = psycopg2.connect(
            dbname=config.DATABASE['DBNAME'],
            host=config.DATABASE['HOST'],
            user=config.DATABASE['USER'],
            password=config.DATABASE['PASSWORD'],
            port=config.DATABASE['PORT'],
        )

engine = create_engine(
            f"postgresql://{config.DATABASE['USER']}:"
            f"{config.DATABASE['PASSWORD']}@"
            f"{config.DATABASE['HOST']}:"
            f"{config.DATABASE['PORT']}/"
            f"{config.DATABASE['DBNAME']}"
        )

In [4]:
"""
all_ids = []
with gzip.open(config.TWEETS_PATH, 'rb') as f:
    for row_number, current_tweet in enumerate(f):
        all_ids.append(json.loads(current_tweet.decode(encoding='utf-8'))['id'])

print(f'Unique id amount: {len(set(all_ids))}, out of total id amount: {len(all_ids)}')
"""

"\nall_ids = []\nwith gzip.open(config.TWEETS_PATH, 'rb') as f:\n    for row_number, current_tweet in enumerate(f):\n        all_ids.append(json.loads(current_tweet.decode(encoding='utf-8'))['id'])\n\nprint(f'Unique id amount: {len(set(all_ids))}, out of total id amount: {len(all_ids)}')\n"

In [5]:
batch_size = 100000

data_rows = []
conversations_existing_ids = utilities.run_written_query('SELECT id FROM conversations', to_dataframe=True, option='from_string').id.astype(str).values
with gzip.open(config.TWEETS_PATH, 'rb') as f:
    for row_number, current_tweet in enumerate(f):
        data_rows.append(json.loads(current_tweet.decode(encoding='utf-8')))
        if (row_number+1) % batch_size == 0:
            print(row_number+1)
            break

/home/xschon/Desktop/PDT/Z1/utils/utilities.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, con=conn)


100000


In [6]:
tweets = pd.DataFrame(data_rows)
metrics = pd.DataFrame(tweets.public_metrics.to_list())
tweets[metrics.columns] = metrics
tweets.rename(columns={'text' : 'content',
                       'lang' : 'language',

                    }, inplace=True)

tweets.columns

Index(['author_id', 'conversation_id', 'created_at', 'entities', 'id',
       'language', 'possibly_sensitive', 'public_metrics', 'referenced_tweets',
       'reply_settings', 'source', 'content', 'context_annotations',
       'attachments', 'in_reply_to_user_id', 'geo', 'withheld', 'like_count',
       'quote_count', 'reply_count', 'retweet_count'],
      dtype='object')

In [7]:
conversations_table = tweets[['id', 'author_id', 'content', 'possibly_sensitive', 'language',
        'source', 'retweet_count', 'reply_count', 'like_count', 'quote_count', 'created_at']]

In [198]:
# Find all referenced tweets - iterate over existing array of references for each tweet
refs = tweets.dropna(subset='referenced_tweets')[['id', 'referenced_tweets']]
conversation_references_table = pd.DataFrame(columns=['conversation_id', 'parent_id', 'type'])

for i in range(refs.referenced_tweets.apply(lambda x : len(x)).max()):
    # Select all references from given layer and find tweets they reffer to
    layer_references = refs.referenced_tweets.apply(lambda x : x[i] if(len(x) > i) else None).dropna()
    conv_refs = pd.DataFrame(layer_references.to_list(), index=layer_references.index)
    conv_refs.rename(columns={'id' : 'parent_id'}, inplace=True)
    conv_refs = (conv_refs.join(refs.id).rename(columns={'id' : 'conversation_id'}))

    conversation_references_table = pd.concat([conversation_references_table, conv_refs])

74074
17
